# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors
spark = SparkSession.builder.appName('lr_consulting').getOrCreate()

In [25]:
data = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)
data.head(1)[0]

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
# Change this to use oneHot encoding
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_values")
dataPrepped = indexer.fit(data).transform(data)
dataPrepped.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_values: double (nullable = false)



In [37]:
from pyspark.sql.window import Window
w = Window.orderBy('Ship_name') 
dataIndexed = dataPrepped.withColumn("index", row_number().over(w))
dataIndexed.sort('index').show()

+-------------+----------------+---+-----------------+----------+------+------+-----------------+-----+------------------+-----+
|    Ship_name|     Cruise_line|Age|          Tonnage|passengers|length|cabins|passenger_density| crew|Cruise_line_values|index|
+-------------+----------------+---+-----------------+----------+------+------+-----------------+-----+------------------+-----+
|    Adventure| Royal_Caribbean| 12|            138.0|     31.14|  10.2| 15.57|            44.32|11.85|               0.0|    1|
|      Allegra|           Costa| 21|            28.43|      8.08|  6.16|   4.1|            35.19|  4.0|               5.0|    2|
|    Amsterdam|Holland_American| 13|             61.0|      13.8|   7.8|  6.88|             44.2|  6.0|               3.0|    3|
|      Arcadia|             P&O|  9|             85.0|     19.68|  9.35|  9.84|            43.19| 8.69|               9.0|    4|
|        Aries|            Star| 22|            3.341|      0.66|   2.8|  0.33|            50.62|

In [12]:
desired_features = (['Age', 'Tonnage', 'passengers','length','cabins','passenger_density','Cruise_line_values'])
desired_features

['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_line_values']

In [39]:
assembler = VectorAssembler(inputCols=desired_features,outputCol='features')
model_input = assembler.transform(dataIndexed)
model_input.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_values: double (nullable = false)
 |-- index: integer (nullable = true)
 |-- features: vector (nullable = true)



In [62]:
train_data,test_data = model_input.randomSplit([0.7,0.3],seed=42)

In [63]:
train_data.describe().show()

+-------+---------+-----------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|          Tonnage|        passengers|            length|           cabins|passenger_density|              crew|Cruise_line_values|            index|
+-------+---------+-----------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+
|  count|      120|        120|               120|              120|               120|               120|              120|              120|               120|               120|              120|
|   mean|     null|       null|15.741666666666667|71.30205000000002|18.541166666666673| 8.161916666666663| 8.87966666666667|         39.96025| 7.836250000000005|               4.8|79.05833333333334|
| std

In [64]:
test_data.describe().show()

+-------+---------+-----------+------------------+-----------------+------------------+------------------+----------------+-----------------+------------------+------------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|          Tonnage|        passengers|            length|          cabins|passenger_density|              crew|Cruise_line_values|            index|
+-------+---------+-----------+------------------+-----------------+------------------+------------------+----------------+-----------------+------------------+------------------+-----------------+
|  count|       38|         38|                38|               38|                38|                38|              38|               38|                38|                38|               38|
|   mean| Infinity|       null|15.526315789473685|71.22978947368422|18.192894736842106| 8.031842105263156|8.67315789473684|39.71368421052631| 7.661315789473684| 5.894736842105263|80.89473684210526|
| stddev| 

In [65]:
lr = LinearRegression(featuresCol='features',labelCol='crew')
lr_model = lr.fit(train_data)

In [66]:
lr_results = lr_model.evaluate(test_data)

In [73]:
lr_results.predictions.show()

+-------------+----------------+---+------------------+----------+------+------+-----------------+-----+------------------+-----+--------------------+------------------+
|    Ship_name|     Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density| crew|Cruise_line_values|index|            features|        prediction|
+-------------+----------------+---+------------------+----------+------+------+-----------------+-----+------------------+-----+--------------------+------------------+
|      Allegra|           Costa| 21|             28.43|      8.08|  6.16|   4.1|            35.19|  4.0|               5.0|    2|[21.0,28.43,8.08,...|  4.08899693096193|
|    Amsterdam|Holland_American| 13|              61.0|      13.8|   7.8|  6.88|             44.2|  6.0|               3.0|    3|[13.0,61.0,13.8,7...|  6.56896710699492|
|      Arcadia|             P&O|  9|              85.0|     19.68|  9.35|  9.84|            43.19| 8.69|               9.0|    4|[9.0,85.0,19.68,9...|

In [67]:
lr_results.rootMeanSquaredError

0.6701835999203916

In [68]:
lr_results.r2

0.9719249282421449

In [69]:
lr_results.meanAbsoluteError

0.5522351972905482

## Boris' assignment
The client accepts error within 0.2. How often are we within 0.2 of the actual

You can automatically accept all 95% or greater process

- Figure out how to link results with original data
- Figure out the error for each
- Provide result